In [4]:
!pip install qiskit qiskit_machine_learning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.1/263.1 kB 12.5 MB/s eta 0:00:00


In [26]:
from qiskit.circuit.library import zz_feature_map, real_amplitudes
from qiskit_machine_learning.algorithms import VQC
from qiskit_machine_learning.optimizers import COBYLA
from qiskit.primitives import StatevectorSampler
import numpy as np

# 1. Prepare dummy data (2 features, binary classification)
num_features = 2
train_features = np.array([[0.1, 0.5], [0.9, 0.2], [0.2, 0.8], [0.8, 0.1]])
train_labels = np.array([0, 1, 0, 1])

# We need test labels to calculate accuracy
test_features = np.array([[0.15, 0.45], [0.85, 0.15]])
test_labels = np.array([0, 1])

# 2. Define the Quantum components
# Feature Map: Encodes classical data into a quantum state
feature_map = zz_feature_map(feature_dimension=num_features, reps=1)

# Ansatz: The "brain" of the circuit with trainable parameters
ansatz = real_amplitudes(num_qubits=num_features, reps=1)


# 4. Define a Callback
loss_values = []
def callback_fn(weights, obj_value):
    loss_values.append(obj_value)
    print(f"Iteration {len(loss_values)}: Loss = {obj_value:.4f}")

# 3. Initialize the VQC
# We use a Sampler primitive to run the circuits
vqc = VQC(
    feature_map=feature_map,
    ansatz=ansatz,
    loss="cross_entropy",
    optimizer=COBYLA(maxiter=20),
    callback=callback_fn,
    sampler=StatevectorSampler()
)

# 4. Train the model
vqc.fit(train_features, train_labels)

# 5. Predict on new data
test_features = np.array([[0.15, 0.45], [0.85, 0.15]])
predictions = vqc.predict(test_features)

# 7. Predict and Print Circuit
print("\n--- Predictions ---")
print(f"Predictions: {vqc.predict(test_features)}")

# Calculating the accuracy score
train_score = vqc.score(train_features, train_labels)
test_score = vqc.score(test_features, test_labels)

print(f"Training Accuracy: {train_score * 100:.1f}%")
print(f"Testing Accuracy:  {test_score * 100:.1f}%")

print("\n--- Circuit Diagram ---")
# Since it's a plain QuantumCircuit now, we just draw it
print(vqc.circuit.draw(output='text'))

Iteration 1: Loss = 1.1840
Iteration 2: Loss = 0.9312
Iteration 3: Loss = 1.0437
Iteration 4: Loss = 1.2540
Iteration 5: Loss = 0.9964
Iteration 6: Loss = 1.0521
Iteration 7: Loss = 0.9558
Iteration 8: Loss = 1.0321
Iteration 9: Loss = 0.9830
Iteration 10: Loss = 0.9992
Iteration 11: Loss = 1.0141
Iteration 12: Loss = 0.9899
Iteration 13: Loss = 0.9586
Iteration 14: Loss = 1.0389
Iteration 15: Loss = 0.9804
Iteration 16: Loss = 0.9767
Iteration 17: Loss = 0.9728
Iteration 18: Loss = 0.9818
Iteration 19: Loss = 0.9976
Iteration 20: Loss = 1.0197

--- Predictions ---
Predictions: [0 0]
Training Accuracy: 50.0%
Testing Accuracy:  50.0%

--- Circuit Diagram ---
     ┌───┐┌───────────┐                                          ┌──────────┐»
q_0: ┤ H ├┤ P(2*x[0]) ├──■────────────────────────────────────■──┤ Ry(θ[0]) ├»
     ├───┤├───────────┤┌─┴─┐┌──────────────────────────────┐┌─┴─┐├──────────┤»
q_1: ┤ H ├┤ P(2*x[1]) ├┤ X ├┤ P((-π + x[0])*(-π + x[1])*2) ├┤ X ├┤ Ry(θ[1]) ├»
     └───┘└───────